# Formation Control test

In [ ]:
%load_ext autoreload
%autoreload 2

Formation control does not rely on the preconstructed roadmap.  
Used to compare trajectory length only

In [ ]:
num_agents = 500
agent_radius = 0.1

# Map config
map_type = "corridor"
num_samples = 300

starts_weight = [0.5, 0.5]
goals_weight = [0.2, 0.8]

In [ ]:
# Load map and instance
import os
import pickle

from swarm_prm.utils.agent_assignment import get_agent_assignment

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_count = get_agent_assignment(num_agents, starts_weight)
goals_agent_count = get_agent_assignment(num_agents, goals_weight)

print(starts_agent_count, goals_agent_count)

In [ ]:
from swarm_prm.solvers.macro import FormationControlSovler

solver = FormationControlSovler(gaussian_prm, agent_radius, starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
                   num_agents=num_agents, time_limit=120)
solution = solver.solve()

assert solution["success"], "sovler failed."


In [ ]:
# Plot solution

import matplotlib.pyplot as plt

fig, ax = gaussian_prm.visualize_roadmap()

# Plot convex sets

x, y = solution["start_poly"].exterior.xy
ax.fill(x, y, color='red', alpha=0.4)
x, y = solution["goal_poly"].exterior.xy
ax.fill(x, y, color='blue', alpha=0.4)

for convex_node in solution["convex_nodes"]:
    x, y = convex_node.poly.exterior.xy
    ax.fill(x, y, color='green', alpha=0.4)

# Plot intermediate Gaussians 
intermediate_nodes = [solution["g_nodes"][i] for i in solution["paths"][0][1:-1]]

for g_node in intermediate_nodes:
    g_node.visualize(ax)

plt.show()

In [ ]:
from swarm_prm.solvers.micro import EvaluationSolver
from swarm_prm.utils import paths_to_macro


g_nodes = solution["g_nodes"]
paths = solution["paths"]
macro_sol = paths_to_macro(paths)
timestep = len(paths[0])-1
starts_idx = solution["starts_idx"]
goals_idx = solution["goals_idx"]

solver = EvaluationSolver(g_nodes, macro_sol, timestep, num_agents, 
                          starts_idx, goals_idx, starts_agent_count, goals_agent_count)

single_agent_paths, cost = solver.solve()

In [ ]:
# Visualize solution
fig, ax = gaussian_prm.visualize_roadmap()

cmap = plt.get_cmap("rainbow")
colors = [cmap(i/ num_agents) for i in range(num_agents)]

for i, path in enumerate(single_agent_paths):
    x = [loc[0] for loc in path]
    y = [loc[1] for loc in path]
    ax.plot(x, y, "-", color=colors[i], linewidth=0.3, alpha=0.8)

plt.show()